In [2]:
import numpy as np
import scipy
from sklearn.preprocessing import MinMaxScaler

dataset = np.loadtxt("/Users/Keshavamurthy/allfeatures1.txt", delimiter=" ");
print(dataset.shape)

X = dataset[:, :33]
Y = dataset[:, 34]
botnetDataset = dataset[dataset[:,34]==1, :]
normalDataset = dataset[dataset[:,34]==0, :]

botnetX = botnetDataset[:,:33]
botnetY = botnetDataset[:, 34]

normalX = normalDataset[:,:33]
normalY = normalDataset[:, 34]

scaler = MinMaxScaler(feature_range=(0, 1))
rescaledBotnetX = scaler.fit_transform(botnetX)
rescaledNormalX = scaler.fit_transform(normalX)
rescaledX = scaler.fit_transform(X)


(52580, 35)


In [3]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model = model.fit(rescaledX, Y)

model.score(rescaledX, Y)

0.92651198174210725

In [14]:
def GetNextBotnetSet(rescaledX, Y):
    step = 4000
    prevStep = 0
    while step < rescaledX.shape[0]:
        yield rescaledX[prevStep:step, :], Y[prevStep:step]
        prevStep = step
        step = step + 4000
        
    return rescaledX[prevStep:, :], Y[prevStep:]

In [23]:
from sklearn.cross_validation import train_test_split

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    model2 = LogisticRegression()
    model2 = model2.fit(X_train, Y_train)
    print(model2.score(X_test, Y_test))

(48479, 33)   (48479,)
0.693399136336
0.799506477483
0.830351634793
0.729179518816
0.867982726712
0.864898210981
0.867982726712
0.868599629858
0.869216533004
0.867982726712
0.8130783467
0.867365823566


In [27]:
from sklearn.cross_validation import cross_val_score
from sklearn import metrics

for botX, botY in GetNextBotnetSet(rescaledBotnetX, botnetY):
    X = np.concatenate((botX, rescaledNormalX))
    Y = np.concatenate((botY, normalY))
    scores = cross_val_score(LogisticRegression(), X, Y, scoring='accuracy', cv=10)
    print(scores.mean())
    predicted = cross_val_predict(LogisticRegression(), X, Y, cv=10)
    print(metrics.accuracy_score(Y, predicted))

(48479, 33)   (48479,)
0.662898722808
0.805944041041
0.838544092798
0.720054954256
0.871869205827
0.873720753223
0.880757942488
0.875201778021
0.879277069918
0.875572605075
0.829664946492
0.870638443622
